# Steane Code Logical T1 Calculation

If there is a file import error make sure you are in the correct path

In [ ]:
import sys
sys.path.append('..')   # the `general_qec` package sits above us

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from general_qec.qec_helpers import *
from general_qec.gates import *
from general_qec.errors import *
from circuit_specific.realistic_steane import *
from circuit_specific.steane_helpers import *

# using datetime module
import datetime # used to see how long things take here

# For fitting exponentials
def exp_decay(x, a, b):
    return a * np.exp(-b * x)

# for exponential warnings
import warnings
#suppress warnings
warnings.filterwarnings('ignore')

## Contents
1. [Introduction](#introduction)
2. [Implementing the Steane code with relaxation and dephasing errors](#errors)
3. [Calculating the logical T1 for the Steane code bit correction circuit](#t1)
4. [More realistic error implementation](#realistic)

## 1. Introduction <a id='introduction'></a>

In this file we will restrict our Steane code connectivity to a line connected circuit and implement our T1 and T2 error model to check for our logical T1 and T2 after running our 3 qubit correction circuit with line connectivity many times. We will also introduce depolarizing errors after we have implemented the T1 and T2 model.

# 2. Implementing the Steane code with Relaxation and dephasing errors  <a id='errors'></a>

#### We will now implement the relaxation and dephasing model created in 05. Error Models.

Since we implemented our Steane code using functions, we will need to pull those in here but change them up a bit.

In [ ]:
# Remember our stabilizer operators? 
# We will need to make a few changes to them and implement the CNOT gates one at a time on our density matrix 
# just like we did with our line connectivity operations

# Define the Stabilizer Operators as CNOT gates 
# (remember that the non-adj CNOT calculation is using line connectivity)
K1_line_operation = np.dot(CNOT(7, 3, 10), np.dot(CNOT(7, 4, 10), np.dot(
    CNOT(7, 5, 10), CNOT(7, 6, 10))))
K2_line_operation = np.dot(CNOT(8, 0, 10), np.dot(CNOT(8, 2, 10), np.dot(
    CNOT(8, 4, 10), CNOT(8, 6, 10))))
K3_line_operation = np.dot(CNOT(9, 1, 10), np.dot(CNOT(9, 2, 10), np.dot(
    CNOT(9, 5, 10), CNOT(9, 6, 10))))

K4_line_operation = np.dot(CZ(7, 3, 10), np.dot(CZ(7, 4, 10), np.dot(
    CZ(7, 5, 10), CZ(7, 6, 10))))
K5_line_operation =np.dot(CZ(8, 0, 10), np.dot(CZ(8, 2, 10), np.dot(
    CZ(8, 4, 10), CZ(8, 6, 10))))
K6_line_operation =np.dot(CZ(9, 1, 10), np.dot(CZ(9, 2, 10), np.dot(
    CZ(9, 5, 10), CZ(9, 6, 10))))

First we can initialize the state of our system and turn it into its density matrix form

In [ ]:
zero = np.array([1, 0])
one = np.array([0, 1])

# the parameters of our system
t1 = 200 * 10**-6 # 200 us - the initial T1
t2 = 150 * 10**-6 # 150 us - Make sure this is less than or equal to T1
tg = 20 * 10**-9 # 20 ns

# t1 = t2 = 1
# Set the initial states of your physical qubits
# initial_state = np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, np.kron(zero, zero))))))
initial_state = np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, np.kron(one, one))))))
# initial_state = 1/np.sqrt(2) * np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#     np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(np.array([[1,1]]), np.kron(
#         np.array([[1,1]]), np.array([[1,1]])))))))

# Set the initial state of your ancilla qubits
ancilla = np.kron(zero, np.kron(zero, zero))

# couple our ancilla qubits
initial_state = np.kron(initial_state, ancilla)

# output our initial state after coupling to the ancilla
print('initial state')
print_state_info(initial_state, 10)

# find the density matrix
initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)

First we will apply the Z correction protocol

In [ ]:
# probability of the state measurments from the density matrix are defined as Tr(p*rho)

# initial_state = collapse_dm(initial_rho)
# print('Initial state before Z correction:')
# print_state_info(initial_state, 10)
# print(' - ')

### Implements the 7 Qubit Steane phase correction code using line connectivity

# - - - - - - - - - - # Z Error Correction # - - - - - - - - - - #
# apply the first hadamard to the ancillas
ancilla_hadamard = np.kron(np.identity(2**7), np.kron(hadamard, np.kron(hadamard, hadamard)))
current_rho = np.dot(ancilla_hadamard, np.dot(initial_rho, ancilla_hadamard.conj().T))
current_rho = rad_error(current_rho, t1, t2, tg)

# apply the control stabilizer gates to current_rho

# apply K1 first:
current_rho = line_rad_CNOT(current_rho, 7, 3, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 7, 4, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 7, 5, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 7, 6, t1, t2, tg, form = 'rho')

# apply K2:
current_rho = line_rad_CNOT(current_rho, 8, 0, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 8, 2, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 8, 4, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 8, 6, t1, t2, tg, form = 'rho')

# apply K3:
current_rho = line_rad_CNOT(current_rho, 9, 1, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 9, 2, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 9, 5, t1, t2, tg, form = 'rho')
current_rho = line_rad_CNOT(current_rho, 9, 6, t1, t2, tg, form = 'rho')

# apply the second hadamard to the ancillas
current_rho = np.dot(ancilla_hadamard, np.dot(current_rho, ancilla_hadamard.conj().T))
current_rho = rad_error(current_rho, t1, t2, tg)

In [ ]:
# Measurement operators for individual qubits
zero_meas = np.kron(zero, zero[np.newaxis].conj().T)
one_meas = np.kron(one, one[np.newaxis].conj().T)

# Define the measurement projection operators
M1 = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(zero_meas, zero_meas)))
M2 = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(zero_meas, one_meas)))
M3 = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(one_meas, zero_meas)))
M4 = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(one_meas, one_meas)))
M5 = np.kron(np.identity(2**7), np.kron(one_meas, np.kron(zero_meas, zero_meas)))
M6 = np.kron(np.identity(2**7), np.kron(one_meas, np.kron(zero_meas, one_meas)))
M7 = np.kron(np.identity(2**7), np.kron(one_meas, np.kron(one_meas, zero_meas)))
M8 = np.kron(np.identity(2**7), np.kron(one_meas, np.kron(one_meas, one_meas)))

all_meas = np.array([M1, M2, M3, M4, M5, M6, M7, M8])

# find the probability to measure each case
m1_prob = np.trace(np.dot(M1.conj().T, np.dot(M1, current_rho)))
m2_prob = np.trace(np.dot(M2.conj().T, np.dot(M2, current_rho)))
m3_prob = np.trace(np.dot(M3.conj().T, np.dot(M3, current_rho)))
m4_prob = np.trace(np.dot(M4.conj().T, np.dot(M4, current_rho)))
m5_prob = np.trace(np.dot(M5.conj().T, np.dot(M5, current_rho)))
m6_prob = np.trace(np.dot(M6.conj().T, np.dot(M6, current_rho)))
m7_prob = np.trace(np.dot(M7.conj().T, np.dot(M7, current_rho)))
m8_prob = np.trace(np.dot(M8.conj().T, np.dot(M8, current_rho)))

all_probs = np.array([m1_prob, m2_prob, m3_prob, m4_prob, m5_prob, m6_prob, m7_prob, m8_prob])

# find which measurement operator is measured based on their probabilities
index = random.choices(all_probs, weights=all_probs, k=1)
index = np.where(all_probs == index)[0][0]

# apply correct measurement collapse of the density matrix
rho_prime = np.dot(all_meas[index], np.dot(current_rho, all_meas[index].conj().T))/(all_probs[index])
# Create our new density matrix after collapsing ancilla qubits

# apply an error for time taken to collapse ancilla
rho = rad_error(rho_prime, t1, t2, tg)

# How many total qubits are in our vector representation
n = int(np.log(len(rho))/np.log(2))

# Measure the three ancilla qubits
# Applying the Z gate operation on a specific qubit based on ancilla
# bits = vector_state_to_bit_state(collapse_dm(rho), 10)[0][0]

In [ ]:
### Just so we can look at the measurement bits that we just collapsed to just now
probs = np.array([])
for i in range(len(rho)):
    probs = np.append(probs, rho[i,i])

collapsed_state = collapse_ancilla(np.sqrt(probs), 10)
bits = vector_state_to_bit_state(collapsed_state, 10)[0][0]

In [ ]:
# find index
m_one = 0
m_two = 0
m_three = 0
if bits[7] == '1':
    m_one = 1
if bits[8] == '1':
    m_two = 1
if bits[9] == '1':
    m_three = 1

# Which qubit do we perform the Z gate on
index = (m_one * 2**2) + (m_three * 2**1) + (m_two * 2**0) - 1

# if no error occurs we dont need to apply a correction
if index == -1:
    final_rho = rho
else:
    # apply the z gate depending on index
    operation = np.kron(np.identity(2**(index)), np.kron(sigma_z, np.kron(
        np.identity(2**(n-3-index-1)), np.identity(2**3))))
    
    final_rho = np.dot(operation, np.dot(rho, operation.conj().T))
    final_rho = rad_error(final_rho, t1, t2, tg) # apply an error for correction gate time

In [ ]:
# total gates applied when implementing the Z correction operators
total_gates_z = 1 + CNOT_gate_tot(7, 3) + CNOT_gate_tot(7, 4) + CNOT_gate_tot(7, 5) + 1 + CNOT_gate_tot(
    8, 0) + CNOT_gate_tot(8, 2) + CNOT_gate_tot(8, 4) + CNOT_gate_tot(8, 6) + CNOT_gate_tot(
    9, 1) + CNOT_gate_tot(9, 2) + CNOT_gate_tot(9, 5) + CNOT_gate_tot(9, 6) + 1 + 2

print('Number of gates applied: ', total_gates_z)
time_z = total_gates_z * tg
tot_time = time_z
print('Time in operation: ', time_z, 'sec')
print('Total time in circuit: ',tot_time , 'sec')

In [ ]:
# Reset the ancilla qubits
M = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(zero_meas, zero_meas)))
meas_prob = np.trace(np.dot(M.conj().T, np.dot(M, final_rho)))
reset_rho = np.dot(M, np.dot(final_rho, M.conj().T))/(meas_prob)
initial_rho = reset_rho

Now lets apply the X error correction protocol

In [ ]:
# - - - - - - - - - - # X Error Correction # - - - - - - - - - - #

# apply the first hadamard to the ancillas
ancilla_hadamard = np.kron(np.identity(2**7), np.kron(hadamard, np.kron(hadamard, hadamard)))
current_rho = np.dot(ancilla_hadamard, np.dot(initial_rho, ancilla_hadamard.conj().T))
current_rho = rad_error(current_rho, t1, t2, tg)

# apply the control stabilizer gates to current_rho

# apply K4 first:
current_rho = line_rad_CZ(current_rho, 7, 3, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 7, 4, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 7, 5, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 7, 6, t1, t2, tg, form = 'rho')

# apply K5:
current_rho = line_rad_CZ(current_rho, 8, 0, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 8, 2, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 8, 4, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 8, 6, t1, t2, tg, form = 'rho')

# apply K6:
current_rho = line_rad_CZ(current_rho, 9, 1, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 9, 2, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 9, 5, t1, t2, tg, form = 'rho')
current_rho = line_rad_CZ(current_rho, 9, 6, t1, t2, tg, form = 'rho')

# apply the second hadamard to the ancillas
current_rho = np.dot(ancilla_hadamard, np.dot(current_rho, ancilla_hadamard.conj().T))
current_rho = rad_error(current_rho, t1, t2, tg)

In [ ]:
# Masurement operators for individual qubits
zero_meas = np.kron(zero, zero[np.newaxis].conj().T)
one_meas = np.kron(one, one[np.newaxis].conj().T)

# Define the measurement projection operators
M1 = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(zero_meas, zero_meas)))
M2 = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(zero_meas, one_meas)))
M3 = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(one_meas, zero_meas)))
M4 = np.kron(np.identity(2**7), np.kron(zero_meas, np.kron(one_meas, one_meas)))
M5 = np.kron(np.identity(2**7), np.kron(one_meas, np.kron(zero_meas, zero_meas)))
M6 = np.kron(np.identity(2**7), np.kron(one_meas, np.kron(zero_meas, one_meas)))
M7 = np.kron(np.identity(2**7), np.kron(one_meas, np.kron(one_meas, zero_meas)))
M8 = np.kron(np.identity(2**7), np.kron(one_meas, np.kron(one_meas, one_meas)))

all_meas = np.array([M1, M2, M3, M4, M5, M6, M7, M8])

# find the probability to measure each case
m1_prob = np.trace(np.dot(M1.conj().T, np.dot(M1, current_rho)))
m2_prob = np.trace(np.dot(M2.conj().T, np.dot(M2, current_rho)))
m3_prob = np.trace(np.dot(M3.conj().T, np.dot(M3, current_rho)))
m4_prob = np.trace(np.dot(M4.conj().T, np.dot(M4, current_rho)))
m5_prob = np.trace(np.dot(M5.conj().T, np.dot(M5, current_rho)))
m6_prob = np.trace(np.dot(M6.conj().T, np.dot(M6, current_rho)))
m7_prob = np.trace(np.dot(M7.conj().T, np.dot(M7, current_rho)))
m8_prob = np.trace(np.dot(M8.conj().T, np.dot(M8, current_rho)))

all_probs = np.array([m1_prob, m2_prob, m3_prob, m4_prob, m5_prob, m6_prob, m7_prob, m8_prob])

# find which measurement operator is measured based on their probabilities
index = random.choices(all_probs, weights=all_probs, k=1)
index = np.where(all_probs == index)[0][0]

# apply correct measurement collapse of the density matrix
rho_prime = np.dot(all_meas[index], np.dot(current_rho, all_meas[index].conj().T))/(all_probs[index])
# Create our new density matrix after collapsing ancilla qubits

# apply an error for time taken to collapse ancilla
rho = rad_error(rho_prime, t1, t2, tg)

# How many total qubits are in our vector representation
n = int(np.log(len(rho))/np.log(2))


In [ ]:
### Just so we can look at the measurement bits that we just collapsed to just now
probs = np.array([])
for i in range(len(rho)):
    probs = np.append(probs, rho[i,i])

collapsed_state = collapse_ancilla(np.sqrt(probs), 10)
bits = vector_state_to_bit_state(collapsed_state, 10)[0][0]

In [ ]:
# find index
m_four = 0
m_five = 0
m_six = 0
if bits[7] == '1':
    m_four = 1
if bits[8] == '1':
    m_five = 1
if bits[9] == '1':
    m_six = 1

# Which qubit do we perform the Z gate on
index = (m_four * 2**2) + (m_six * 2**1) + (m_five * 2**0) - 1

# if no error occurs we dont need to apply a correction
if index == -1:
    final_rho = rho
else:
    # apply the z gate depending on index
    operation = np.kron(np.identity(2**(index)), np.kron(sigma_x, np.kron(
        np.identity(2**(n-3-index-1)), np.identity(2**3))))
    
    final_rho = np.dot(operation, np.dot(rho, operation.conj().T))
    final_rho = qubit_rad_error_matrix(final_rho, t1, t2, tg) # apply an error for correction gate time   

In [ ]:
# check our collapsed density matrix to see if we are in the codespace.
collapsed_rho = collapse_dm(final_rho)  
                                 
print('Final state after steane code:')
print_state_info(collapsed_rho, 10)

In [ ]:
# total gates applied when implementing X correction operators (for every CZ we add 2 for the hadamard converison)
total_gates_x = total_gates_z + (2*12) # CZ gates

print('Number of gates applied: ', total_gates_x)
time_x =  total_gates_x * tg
tot_time += time_x
print('Time in operation: ', time_x, 'sec')
print('Total time in circuit: ', tot_time, 'sec')

In [ ]:
# ct stores current time
ct = datetime.datetime.now()
print('Start Time: ', ct)
print_state_info(collapse_dm(rho), 10)
ct = datetime.datetime.now()
print('End Time: ', ct)

## 3. Calculating the logical T1 for the Steane code  <a id='t1'></a>

We will change parameters and loop over the number of iterations that we impelemnt the circuit

In [ ]:
zero = np.array([1, 0])
one = np.array([0, 1])

# the parameters of our system
t1 = 200 * 10**-6 # 200 us - the initial T1
t2 = 150 * 10**-6 # 150 us - Make sure this is less than or equal to T1
tg = 20 * 10**-9 # 20 ns
# t1 = t2 = 1
spam_prob = None
qubit_error_probs = None

# Set the initial states of your physical qubits
initial_psi = one
initial_state = np.kron(initial_psi, np.kron(initial_psi, np.kron(initial_psi, np.kron(
    initial_psi, np.kron(initial_psi, np.kron(initial_psi, initial_psi))))))

ideal_state = initialize_steane_logical_state(initial_state)

initial_state = np.kron(initial_state, np.kron(zero, np.kron(zero, zero)))

initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)

rho = initial_rho

In [ ]:
print('Working on plotting the probability of state measurements overtime...')
# ct stores current time
ct = datetime.datetime.now()
print("start time:", ct)

# Masurement operators for individual qubits
zero_meas = np.kron(zero, zero[np.newaxis].conj().T)
one_meas = np.kron(one, one[np.newaxis].conj().T)
    
# all_pops = np.array([])
all_pops0 = np.array([])
all_pops1 = np.array([])
other_probs = np.array([])
count = np.array([])
# Apply the circuit  times
iterations = 5  # assume a slow iteration - you need ~30 for a nice plot, etc.
for i in range(iterations):
    count = np.append(count, i)
    rho = realistic_steane(rho, t1=t1, t2=t2, tg=tg, qubit_error_probs=qubit_error_probs, spam_prob=spam_prob)

    # Measurement operator to see if you are in the 0 logical state
    M0 = np.kron(zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(
        zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(
        one_meas, np.kron(zero_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(one_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(
        zero_meas, np.kron(one_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(
        one_meas, np.kron(one_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(
        one_meas, np.kron(one_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(one_meas, np.kron(
        zero_meas, np.kron(one_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(one_meas, np.kron(one_meas, np.kron(one_meas, np.kron(
        one_meas, np.kron(zero_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(
        zero_meas, np.kron(zero_meas, np.kron(one_meas, np.identity(2**3))))))))

    # probability of being in the 0 logical state
    prob0 = np.trace(np.dot(M0.conj().T, np.dot(M0, rho)))

    # Measurement operator to see if you are in the 1 logical state
    M1 = np.kron(one_meas, np.kron(one_meas, np.kron(one_meas, np.kron(one_meas, np.kron(
        one_meas, np.kron(one_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(
        zero_meas, np.kron(one_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(
        one_meas, np.kron(zero_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(one_meas, np.kron(
        zero_meas, np.kron(zero_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(one_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(
        zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(
        one_meas, np.kron(zero_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(
        zero_meas, np.kron(one_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(
        one_meas, np.kron(one_meas, np.kron(zero_meas, np.identity(2**3))))))))

    # probability of being in the 1 logical state
    prob1 = np.trace(np.dot(M1.conj().T, np.dot(M1, rho)))

    # any other probability
    prob_other = 1 - prob0 - prob1

    all_pops0 = np.append(all_pops0, prob0)
    all_pops1 = np.append(all_pops1, prob1)
    other_probs = np.append(other_probs, prob_other)
    
    if i == 0:
        # ct stores current time
        ct = datetime.datetime.now()
        print('Time after 1st iteration: ', ct)
        
# ct stores current time
ct = datetime.datetime.now()
print("end time:", ct)

In [ ]:
## -- Plotting our data and finding a line of best fit -- ##
print('The ideal state of our system:')
print_state_info(ideal_state, 7)
print('- - -')
print('Physical T1: ', t1, ' sec')
print('Physical T2:', t2, ' sec')
print('Gate time (Tg): ', tg, 'sec')
print('Depolarizing error by probability at each qubit: ', qubit_error_probs)
print('SPAM error probability: ', spam_prob )

# Add data to the plot
plt.figure(figsize=(10,4))
plt.scatter(count, all_pops0, s = 5, c = 'cornflowerblue', label = 'Logical |0>')
plt.scatter(count, all_pops1, s = 5, c ='seagreen', label = 'Logical |1>')
plt.scatter(count, other_probs, s = 5, c ='red', label = 'any other state')
plt.title('Qubit Meaurement Probability as a function of running Steane code')
plt.xlabel('Number of Iterations')
plt.ylabel('Probability of Measurement')
plt.axhline(y = 1/np.e, color = 'y', linestyle = 'dotted')
# Find and plot the fitted exponential for the |111> state
# xdata = (count)
# ydata = all_pops1
# popt, pcov = curve_fit(exp_decay, xdata, ydata)
# #     if 0<popt[1]<1:
# plt.plot(xdata, exp_decay(xdata, *popt), 'black', label='fit: a=%5.3f, b=%5.3f' % tuple(popt), linestyle = 'dashed')
# print('- - - - -')
# circuit_runs = 1/popt[1]
# if tg!=None:
#     print('Calculated Circuit iterations until logical failure: ', circuit_runs)
#     print('Calculated Logical T1: ', (((circuit_runs * 29) + 2) * tg), 'sec')
# else:
#     print('Calculated Circuit iterations until logical failure: ', circuit_runs)
# plt.ylim([-0.1, 1.1])
# plt.legend()

plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
print('Note that the fitted line may have errors')
plt.show()

## 4. More realistic error implementation <a id='realistic'></a>

Now we will not apply discrete errors as our error models will implement continuous errors on all gate operations and state preparation and measurement errors.

In [ ]:
zero = np.array([1, 0])
one = np.array([0, 1])

# the parameters of our system
t1 = 200 * 10**-6 # 200 us - the initial T1
t2 = 150 * 10**-6 # 150 us - Make sure this is less than or equal to T1
tg = 20 * 10**-9 # 20 ns

# state preparation and measurement errors
spam_prob = 0.00001

p_q0 = p_q1 = p_q2 = p_q3 = p_q4 = p_q5 = p_q6 = p_q7 = p_q8 = p_q9 = 1e-5
# define your error probability for each qubit
qubit_error_probs = np.array([p_q0, p_q1, p_q2, p_q3, p_q4, p_q5, p_q6, p_q7, p_q8, p_q9])

# Set the initial states of your physical qubits
# Set the initial states of your physical qubits
initial_psi = one
initial_state = np.kron(initial_psi, np.kron(initial_psi, np.kron(initial_psi, np.kron(
    initial_psi, np.kron(initial_psi, np.kron(initial_psi, initial_psi))))))

ideal_state = initialize_steane_logical_state(initial_state)

initial_state = np.kron(initial_state, np.kron(zero, np.kron(zero, zero)))

initial_rho = np.kron(initial_state, initial_state[np.newaxis].conj().T)

rho = realistic_steane(
    initial_rho, t1=t1, t2=t2, tg=tg, qubit_error_probs=qubit_error_probs, spam_prob=spam_prob)

In [ ]:
iterations = 2
print(f"Runing {iterations} iterations...")

In [ ]:
print('Working on plotting the probability of state measurements over time...')
all_pops0 = np.array([])
all_pops1 = np.array([])
other_probs = np.array([])
count = np.array([])
# Masurement operators for individual qubits
zero_meas = np.kron(zero, zero[np.newaxis].conj().T)
one_meas = np.kron(one, one[np.newaxis].conj().T)
# Apply the circuit  times
for i in range(iterations):
    print("iteration", i)
    count = np.append(count, i)
    rho = realistic_steane(
        rho, t1=t1, t2=t2, tg=tg, qubit_error_probs=qubit_error_probs, spam_prob=spam_prob
    )

    # Measurement operator to see if you are in the 0 logical state
    M0 = np.kron(zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(
        zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(
        one_meas, np.kron(zero_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(one_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(
        zero_meas, np.kron(one_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(
        one_meas, np.kron(one_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(
        one_meas, np.kron(one_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(one_meas, np.kron(
        zero_meas, np.kron(one_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(one_meas, np.kron(one_meas, np.kron(one_meas, np.kron(
        one_meas, np.kron(zero_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(
        zero_meas, np.kron(zero_meas, np.kron(one_meas, np.identity(2**3))))))))

    # probability of being in the 0 logical state
    prob0 = np.trace(np.dot(M0.conj().T, np.dot(M0, rho)))

    # Measurement operator to see if you are in the 1 logical state
    M1 = np.kron(one_meas, np.kron(one_meas, np.kron(one_meas, np.kron(one_meas, np.kron(
        one_meas, np.kron(one_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(
        zero_meas, np.kron(one_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(
        one_meas, np.kron(zero_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(one_meas, np.kron(
        zero_meas, np.kron(zero_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(one_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(
        zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(
        one_meas, np.kron(zero_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        one_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(zero_meas, np.kron(
        zero_meas, np.kron(one_meas, np.kron(one_meas, np.identity(2**3)))))))) + np.kron(
        zero_meas, np.kron(zero_meas, np.kron(one_meas, np.kron(zero_meas, np.kron(
        one_meas, np.kron(one_meas, np.kron(zero_meas, np.identity(2**3))))))))

    # probability of being in the 1 logical state
    prob1 = np.trace(np.dot(M1.conj().T, np.dot(M1, rho)))

    # any other probability
    prob_other = 1 - prob0 - prob1

    all_pops0 = np.append(all_pops0, prob0)
    all_pops1 = np.append(all_pops1, prob1)
    other_probs = np.append(other_probs, prob_other)



In [ ]:
## -- Plotting our data and finding a line of best fit -- ##
print('The ideal state of our system:')
print_state_info(ideal_state, 10)
print('- - -')
print('Physical T1: ', t1, ' sec')
print('Physical T2:', t2, ' sec')
print('Gate time (Tg): ', tg, 'sec')
print('Depolarizing error by probability at each qubit: ', qubit_error_probs)
print('SPAM error probability: ', spam_prob )

# Add data to the plot
plt.figure(figsize=(10,4))
plt.scatter(count, all_pops0, s = 1, c = 'cornflowerblue', label = 'Logical |0>')
plt.scatter(count, all_pops1, s = 1, c ='seagreen', label = 'Logical |1>')
plt.scatter(count, other_probs, s = 1, c ='red', label = 'any other state')
plt.title('Qubit Meaurement Probability as a function of running Steane code')
plt.xlabel('Number of Iterations')
plt.ylabel('Probability of Measurement')
plt.axhline(y = 1/np.e, color = 'y', linestyle = 'dotted')
# Find and plot the fitted exponential for the |111> state
xdata = (count)
ydata = all_pops1
popt, pcov = curve_fit(exp_decay, xdata, ydata)
plt.plot(xdata, exp_decay(xdata, *popt), 'black', label='fit: a=%5.3f, b=%5.3f' % tuple(popt), linestyle = 'dashed')
print('- - - - -')
circuit_runs = 1/popt[1]
if tg!=None:
    print('Calculated Circuit iterations until logical failure: ', circuit_runs)
    print('Calculated Logical T1: ', tot_time, 'sec')
else:
    print('Calculated Circuit iterations until logical failure: ', circuit_runs)
plt.ylim([-0.1, 1.1])
plt.legend()

plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
print('Note that the fitted line may have errors')
plt.show()
